<a href="https://colab.research.google.com/github/Diogo-code/Data-Mining-Diogo/blob/main/OneHotEncodingAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model as lm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

university_towns = []

with open('/content/university_towns.txt') as file:
  for line in file:
    if '[edit]' in line:
      state = line 
    else:
      university_towns.append((state, line))

university_towns[:10]

towns_df = pd.DataFrame(university_towns, columns=['State', 'RegionName'])



#towns_df['State'] = towns_df['State'].str.replace('\[edit\]\\n', '')

def get_citystate(item):
  if ')' in item:
    return item[:item.find(')')+1]
  elif '[' in item:
    return item[:item.find('[')]
  else:
    return item




towns_df = towns_df.applymap(get_citystate)
new = towns_df['RegionName'].str.split('(',n = 1, expand = True)

towns_df['City'] = new[0]

towns_df['University'] = new[1].str.replace(')','')

towns_df.drop(columns = ['RegionName'], inplace = True)

towns_df['State'] = towns_df['State'].astype(str)
towns_df['City'] = towns_df['City'].astype(str)
towns_df['University'] = towns_df['University'].astype(str)

label_encoder = LabelEncoder()

towns_df['State'] = label_encoder.fit_transform(towns_df['State']) + 1

data_column_category = towns_df.select_dtypes(exclude=[np.number]).columns
#data_column_numeric = towns_df.select_dtypes(include=[np.number]).columns

scalecolumn = np.log10(towns_df['State'])

onehot_encoder = OneHotEncoder(sparse=False)

onehot_encoded = onehot_encoder.fit_transform(towns_df[data_column_category])

onehot_encoded_frame = pd.DataFrame(onehot_encoded, columns = onehot_encoder.get_feature_names(data_column_category))

final_df = pd.concat([scalecolumn ,onehot_encoded_frame, towns_df[data_column_category]], axis=1)

#Features are City and University
#Target column is state is State
#The problem with one hot encoding all data is that all variables in the columns, including the target column will become their own columns with 1 or 0



#Making train test split
DATAFeatures = onehot_encoder.get_feature_names(data_column_category)
TargetFeature = "State"
X_train, X_test, Y_train, Y_test = train_test_split(final_df[DATAFeatures], final_df['State'], test_size=.25)

# Liner regression model in order to predict the target feature
xb = np.array(X_train)
#xb = xb.reshape(-1,1)
yb = np.array(Y_train)
yb = yb.reshape(-1,1)

xb_test = np.array(X_test)
#xb_test = xb.reshape(-1,1)
yb_test = np.array(Y_test)
yb_test = yb_test.reshape(-1,1)

bestModel = lm.LinearRegression().fit(xb,yb)
BMscore = bestModel.score(xb_test, yb_test)

print(final_df.info())
print('R squared:',BMscore)

#There is no difference between using the scaled target and the non-scaled target. The only thing that scale does is that it just represents the target column as smaller than it actually is.
#You can not make accurate predictions using linear regression on this dataset. According to the R-squared value, the target value can not be accurately predicted. This could be because the data is not linear.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Columns: 1005 entries, State to University
dtypes: float64(1003), object(2)
memory usage: 4.0+ MB
None
R squared: -0.2415202235677338
